In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd

In [2]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import csv

database_folder = "/content/drive/MyDrive/DataBase"
f = os.listdir(database_folder)

for s in f:
  training_data = []
  with open('/content/drive/MyDrive/DataBase/'+s, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']

In [5]:
dir_number = 16

angles = np.arctan2(data[:,:,:,:,1], data[:,:,:,:,0])
angles = np.round(((angles*360/(2*np.pi))/(360/dir_number)))+(dir_number/2)
angles = angles % dir_number
directions = [np.where(angles == i , 1, 0) for i in range(1, dir_number+1)]
for i in range(dir_number):
    directions[i] = np.sum(directions[i], axis = 1)
directions = np.array(directions)
direction_data= np.where(directions > 0 , 1, 0)

In [6]:
print(directions.shape)

(16, 1465, 144, 73)


In [7]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]



def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_long(arr[2]),grid_lat(arr[3]),grid_time(arr[4]),int(arr[5])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i,data_winds, data_dirs):
    array = grid_array(training_data[i])
    time = array[4]
    long_start = array[0]
    lat_start = array[1]
    long_end = array[2]
    lat_end = array[3]
    alt = array[5]
    dirs = data_dirs[:,time:time+dtime,:,:]
    dirs = np.transpose(dirs,(1,0,2,3))
    pos = np.zeros((dtime,1,dirs.shape[2],dirs.shape[3]))
    pos[0,0,int(long_end),int(lat_end)] = -data_std*1.3
    pos[0,0,long_start,lat_start] = data_std*1.3
    input = np.concatenate((dirs,pos),axis=1)
    action = np.arctan2(data_winds[time,alt,long_start,lat_start,1],data_winds[time,alt,long_start,lat_start,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/(360/dir_number))%dir_number)

    return (input,action)


In [8]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, wind_data, direction_data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.wind_data = wind_data
        self.direction_data = direction_data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx,self.wind_data, self.direction_data)
        inp = np.pad(inp,((0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [9]:
trainset = CustomDataset(training_data,1,data,direction_data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [10]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)


torch.Size([32, 1, 17, 148, 73])


In [11]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out

class ProjectedResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ProjectedResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 2, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.projector = nn.AvgPool2d(kernel_size, stride = 2 , padding = padding)
        self.Conv1x1 = nn.Conv2d(in_channels, out_channels, 1, 1, 0)
        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x

        identity = self.projector(x)
        identity =  self.Conv1x1(identity)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += identity
        x = self.relu(x)
        x = self.dropout(x)
        return x

In [12]:
class Net(nn.Module):
    def __init__(self , print_shape=False):
        self.print_shape = print_shape
        dropoutr = 0.2
        super(Net, self).__init__()
        self.layers1 = nn.ModuleList( [
        nn.Conv2d(dir_number+1, 64, (3,3)),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(dropoutr),
        ResBlock(64,64, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(64,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(128,256, (3,3), 1, 1, dropoutr),
        ResBlock(256,256, (3,3), 1, 1, dropoutr),
        ResBlock(256,256, (3,3), 1, 1, dropoutr),
        ResBlock(256,256, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(256,512, (3,3), 1, 0, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        nn.Conv2d(512, 512, kernel_size = (2,2), stride = (2,2)),
        nn.BatchNorm2d(512),
        nn.Dropout(dropoutr)])

        self.layers = nn.ModuleList([
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(512, 8)])

        for l in self.layers1:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()


        for l in self.layers:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv2d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:]
        for l in self.layers1:
            x = l(x)
            if  (isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or  isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

In [13]:
net = Net().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-12-b95d2f3993a7>:61: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [14]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
r = net.to(dev)
r = net(a).to(dev)
b = b.to(dev)
z = torch.argmax(torch.softmax(r,1), axis=1).to(dev)
print(torch.softmax(r,1))
print(z)
print(b - z)
# print(r.data)
# print(b)


tensor([[0.2163, 0.1327, 0.0768, 0.0208, 0.1416, 0.0424, 0.1561, 0.2133],
        [0.0348, 0.0856, 0.0147, 0.0871, 0.4754, 0.1758, 0.0376, 0.0889],
        [0.1640, 0.1071, 0.0729, 0.0564, 0.2191, 0.2547, 0.0227, 0.1031],
        [0.0203, 0.2465, 0.1967, 0.0817, 0.0980, 0.0224, 0.1572, 0.1771],
        [0.0232, 0.3105, 0.0559, 0.0668, 0.2606, 0.0278, 0.0290, 0.2262],
        [0.0146, 0.1159, 0.0827, 0.0546, 0.0882, 0.0490, 0.3882, 0.2068],
        [0.0912, 0.0525, 0.0414, 0.0301, 0.0732, 0.1181, 0.0994, 0.4941],
        [0.0336, 0.0546, 0.0560, 0.3310, 0.2965, 0.0951, 0.0824, 0.0509],
        [0.0517, 0.0995, 0.0509, 0.0441, 0.0663, 0.2758, 0.3343, 0.0773],
        [0.0772, 0.2313, 0.0936, 0.0685, 0.3099, 0.0320, 0.1139, 0.0735],
        [0.0103, 0.1056, 0.3201, 0.1892, 0.2926, 0.0091, 0.0022, 0.0708],
        [0.0284, 0.1056, 0.0807, 0.1497, 0.1530, 0.2842, 0.1145, 0.0838],
        [0.1130, 0.3242, 0.0164, 0.0724, 0.0923, 0.0261, 0.2515, 0.1041],
        [0.1568, 0.1422, 0.0446, 0.350

In [15]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [16]:

for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=0)
    for i, data in enumerate(trainloader, 0):
      try:
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 10000 == 9999:    # print every 2000 mini-batches
               torch.save(net, "/content/drive/MyDrive/resnet3.pt")
      except Exception as e:
        print(e)



print('Finished Training')

CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below migh

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(100):
        data = next(iter(trainloader))
        images, labels = data
        images = images.float().to(dev)
        labels = labels.to(dev)
        outputs = net(images)
        predicted = torch.argmax(torch.softmax(outputs.data, 1),axis =1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 75 %
